<a href="https://colab.research.google.com/github/takatakamanbou/ML/blob/2023/ex05notebookB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ML ex05notebookB

<img width=72 src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/ML-logo.png"> [この授業のウェブページ](https://www-tlab.math.ryukoku.ac.jp/wiki/?ML/2023)


----
## ニューラルネットワークと深層学習 (1)
----

**ニューラルネットワーク**(Neural Network)は，ヒトなどの生物の「ニューロン」（神経細胞，Neuron）が互いに繋がり合って作る「回路網」(Network)を指す言葉です（注）．
もともとは，神経細胞や脳の仕組みをまねた数理モデルという色合いがありましたが，近年ではその色は薄れ，実用的な機械学習モデルとして発展してきています．
このようなニューラルネットワークの学習は **深層学習**(Deep Learning) と呼ばれ，いわゆる人工知能(AI)の中核的な技術となっています．何が「深層」，「Deep」なのかは後述します．回帰にも識別にも使うことができます．

<span style="font-size: 75%">
※注: 日本語では「（人工）神経回路網」と言います．
「人工」(Artifitial)と付けない場合，生体内にある本物の神経回路網を指すこともあります．
</span>




----
### 準備



以下，コードセルを上から順に実行してながら読んでいってね．

In [ ]:
# 準備あれこれ
import numpy as np
import matplotlib.pyplot as plt
import seaborn
seaborn.set()

---
### ニューロンのモデル


<img width="40%" src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/neuron3.png" align="right">


ヒトの脳には数百億のニューロンが存在しています．
ニューロンは互いに信号を伝達する機能をもっており，多数のニュー
ロンが相互につながりあって情報処理を行なっています．
生命維持から感情・思考にいたるまでの脳機能は，主にこれらニューロ
ンの集団が担っているものと考えられています．

右図の上段は，実際のニューロンを模式的に描いたもの，中段は，これらニューロン間の信号伝達の様子を表したものです．大きな丸がニューロンを表し，線分がニューロン間のつながりを表します．

ニューロンのふるまいを思い切って単純化すると，次のようにまとめられます．

1. 他のニューロンの出力を受け取る
1. それらの和を計算する．ただし，ニューロン間のつながりの強さに応じて，他のニューロンからやってきた値を重み付けした和を計算する．
1. その重み付け和の値に応じて自分の値を決めて出力する．

このようなニューロンのふるまいは，次式のようにモデル化できます．

$$
y = \sigma\left(w_0 + \sum_{d=1}^{D}w_dx_d\right) \qquad (1)
$$

ただし，$x_d$ は他のニューロンの出力，$y$ は自分の出力です．

$w_d$ は $d$ 番目のニューロンと自分との間のつながりの強さを決める量で，「結合重み」(connection weight)や「重み」(weight)などと呼ばれます．$w_0$は「しきい値」(threshold)または「バイアス項」(bias term)などと呼ばれます．

$\sigma$は，重み付け和の値を変換する役割をする関数です．**活性化関数**(activation function)と呼ばれます．

ニューロンの出力は，他のニューロンからやってくる値が同じでも，重みの値が変わると変化します．つまり，重みがパラメータとなっています．実際の脳内でも，シナプス（神経細胞同士の信号伝達を橋渡しする役割をする）での信号の伝わり方が変化することで記憶や学習が起こっていると考えられています．


活性化関数としては，ロジスティック回帰モデルでも使われている**シグモイド関数**(sigmoid function)の他に，**双曲線正接関数**(hyperbolic tangent function)，**Rectified Linear 関数**（**ReLU関数**ともいいます）などが使われます（注）．

$$
\begin{aligned}
\mbox{Logistic Sigmoid}\quad  & \sigma(s) = \frac{1}{1+{\rm e}^{-s}}\\
\mbox{Hyperbolic Tangent}\quad & \sigma(s) = {\rm tanh}(s) = \frac{{\rm e}^{s} - {\rm e}^{-s}}{{\rm e}^{s} + {\rm e}^{-s}}\\
\mbox{Rectified Linear}\quad & \sigma(s) = \left\{
    \begin{array}{ll}
    s & (s \ge 0)\\
    0 & {\rm otherwise}\\
\end{array} \right.
\end{aligned}
$$

以下にそれぞれのグラフを示します．

<span style="font-size: 75%">
※注: 「シグモイド」は「S字状の」という意味の語なので，双曲線正接関数なども含めて類似した形をしている関数たちの総称です．式(X)のものは，正確には「ロジスティックシグモイド関数」(logistic sigmoid function)といいます．
また，「Rectified Linear」な活性化関数を採用したニューロンを Rectified Linear Unit と呼ぶことから，この活性化関数を「ReLU」と呼ぶことがあります．
</span>

In [ ]:
# 活性化関数の値を計算
xmin, xmax = -4, 4
X = np.linspace(xmin, xmax, num=100)
Y1 = 1/(1+np.exp(-X)) # ロジスティックシグモイド
Y2 = np.tanh(X)       # 双曲線正接
Y3 = np.maximum(X, 0) # ReLU

# グラフに描く
fig = plt.figure(facecolor='white', figsize=(12,4))
ax1 = fig.add_subplot(131)
ax1.set_xlim(xmin, xmax)
ax1.set_ylim(-1.2, 1.2)
ax1.axhline(y=0, color='black', linestyle='-')
ax1.axvline(x=0, color='black', linestyle='-')
ax1.axhline(y=1, color='gray', linestyle='--')
ax1.plot(X, Y1, c='red', linewidth=3, label='Sigmoid')
ax1.legend()
ax2 = fig.add_subplot(132)
ax2.set_xlim(xmin, xmax)
ax2.set_ylim(-1.2, 1.2)
ax2.axhline(y=0, color='black', linestyle='-')
ax2.axvline(x=0, color='black', linestyle='-')
ax2.axhline(y=1, color='gray', linestyle='--')
ax2.axhline(y=-1, color='gray', linestyle='--')
ax2.plot(X, Y2, c='red', linewidth=3, label='tanh(s)')
ax2.legend()
ax3 = fig.add_subplot(133)
ax3.set_xlim(xmin, xmax)
ax3.set_ylim(xmin, xmax)
ax3.axhline(y=0, color='black', linestyle='-')
ax3.axvline(x=0, color='black', linestyle='-')
ax3.plot(X, Y3, c='red', linewidth=3, label='ReLU')
ax3.legend()
plt.show()

このような活性化関数を用いることで，ニューロンが入力に対して非線形な応答をするようになります．詳しくは後述しますが，この非線形性が，ニューラルネットワークの能力の鍵となっています．

---
### ニューラルネットワークの構造

<img width="60%" src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/neuralnet.png" align="right">

ニューラルネットワークとは，上述のようなニューロンが相互に繋がりあったものです．
その繋がり方としては，図の左のように相互に無秩序に繋がるようなものも考えられるのですが，近年のAIで幅広く採用されているのは，右のようにニューロンたちが整然と層を成してならんでいる形のものです．図の薄く色付けされた長方形の箱が一つの層(layer)を表しています．このようなニューラルネットワークのことを，
「階層型ニューラルネットワーク」と呼びます．


階層型ニューラルネットワークでは，情報が入力されると，層から層へと順次伝わっていき，最終的な出力がなされます．
このように情報の伝わり方が一方向であることから，「フィードフォワード（feed-forward, 順伝播）型」と呼ぶこともあります（注）．

<span style="font-size: 75%">
※注: 「多層パーセプトロン」(Multi-Layer Perceptron)という呼び方もあります．
</span>

<img width="40%" src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/neuralnet2.png" align="right">

階層型ニューラルネットワークの一例を右図に示します．このニューラルネットワークの入出力をきちんと式で表してみましょう．このニューラルネットワークが$D$次元の入力を受け取ると，それらはまず**中間層**にある $H$ 個のニューロンへと伝わります．
$d$番目の入力 $x_d$ が中間層の $h$ 番目のニューロンへ伝わる際の重みを $v_{h,d}$ と表すと，このニューロンの出力 $y_h$ は次式のように計算されます．

$$
y_h = \sigma\left( v_{h,0} + \sum_{d=1}^{D}v_{h,d}x_d\right) \qquad (H = 1, 2, \ldots, H) 
$$

次に，これら中間層ニューロンの出力が**出力層**にある $M$ 個のニューロンへと伝わります．
$h$番目の中間層ニューロンの出力 $y_h$ が出力層の $m$ 番目のニューロンへ伝わる際の重みを $w_{m,h}$ と表すと，このニューロンの出力 $z_m$ は次式のように計算されます．

$$
z_m = \sigma\left( w_{m,0} + \sum_{h=1}^{H}w_{m,h}y_h\right)\qquad (M = 1, 2, \ldots, M)
$$

そして，これら出力層ニューロンの値がニューラルネットワークの出力となります．

このニューラルネットワークの場合，学習によって調節されるパラメータは $v_{h,d}$ ($h = 1, 2, \ldots, H, d = 0, 1, 2, \ldots, D$) が $H\times(D+1)$ 個と $w_{m,h}$ ($m=1,2,\ldots,M, h = 1, 2, \ldots, H$) が $M\times(H+1)$ の計 $H(D+1)+M(H+1)$ 個あります．
これらは通常教師あり学習によって調節されます．その方法については後述します．

ここでは中間層が一つだけの例を示しましたが，2層以上の中間層を用いることも可能です．
また，中間層は「隠れ層」と呼ばれることもあります．

<img width="60%" src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/neuralnet3.png" align="right">

式(1)の形からお気づきかもしれませんが，ニューロンひとつは2クラス識別のロジスティック回帰モデルと同じ形をしています．一般のロジスティック回帰モデルは，$K$（クラス数）個のニューロンから成る，一層だけのニューラルネットワークといえます．

図の左側は，そのようなモデルを図に表したものです．下の図のように，赤青2クラスのデータが与えられたとしても，ロジスティック回帰モデルは図の緑色の実線のような直線の識別境界しか作れません．破線のような識別境界が作れるとよいのですが，そのような能力はありません．

しかし，図の右側のように入力と出力の間に中間層を追加したニューラルネットワークにすると，非線形な活性化関数のおかげで，入力を非線形変換したものを中間層で作ることができます．
そのため，中間層でデータが直線で分けられるように非線形変換してから出力層でそれを直線で分けて識別する，ということが可能になります．これを入力の空間で見ると，破線のような識別境界を実現したことになります．

ここでは識別の例で説明しましたが，回帰の問題でも同様のことがいえます．ニューラルネットワークでは，入力と出力の間に中間層を挟むことで，入力データを非線形変換してから処理します．非線形変換によって後に続く処理が容易になり，複雑な問題でも解くことができます．

中間層は，2層以上使うこともできます．中間層を増やすことでより複雑な非線形変換が可能となります．しかし，多くの中間層から成るニューラルネットワークの学習は難しく，計算コストもかかるため，以前（20世紀末ころ）は数層程度が限界でした．それが，研究の進展と，計算機の能力の飛躍的な向上により，近年（2010年ころ以降）では数十層以上あるニューラルネットワークの学習が可能となり，実用レベルの性能が得られるようになりました（注）．「深層」，「Deep」はこのような多層のニューラルネットワークを指す言葉です．

<span style="font-size: 75%">
※注: インターネットの普及により，データを容易にたくさん集められるようになって，大規模なデータで学習させることが可能になった，というのも大きな原動力になりました．
</span>